In [56]:
import pandas as pd
from keybert import KeyBERT
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer


In [2]:
!pip3 install keybert

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 86.0/86.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 238.7/238.7 kB 3.6 MB/s eta 0:00:00
     ---------------------------------------- 84.5/84.5 kB ? eta 0:00:00
     ---------------------------------------- 6.8/6.8 MB 15.4 MB/s eta 0:00:00
     ------------------------------------- 172.3/172.3 MB 11.1 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 12.7 MB/s eta 0:00:00
     ------------------------------------- 977.8/977.8 kB 12.4 MB/s eta 0:00:00
     -------------------------------------- 200.1/200.1 kB 6.1 MB/s eta 0:00:00
     ---------------------------------------- 6.5/6.5 MB 11.8 MB/s eta 0:00:00
     ---

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
kw_model = KeyBERT()
keywords = kw_model.extract_keywords('''Supervised learning is the machine learning task of learning a function that
         maps an input to an output based on example input-output pairs. It infers a
         function from labeled training data consisting of a set of training examples.
         In supervised learning, each example is a pair consisting of an input object
         (typically a vector) and a desired output value (also called the supervisory signal).
         A supervised learning algorithm analyzes the training data and produces an inferred function,
         which can be used for mapping new examples. An optimal scenario will allow for the
         algorithm to correctly determine the class labels for unseen instances. This requires
         the learning algorithm to generalize from the training data to unseen situations in a
         'reasonable' way (see inductive bias).
''', keyphrase_ngram_range=(1, 3))


In [7]:
%%time
keywords = kw_model.extract_keywords('''Supervised learning is the machine learning task of learning a function that
         maps an input to an output based on example input-output pairs. It infers a
         function from labeled training data consisting of a set of training examples.
         In supervised learning, each example is a pair consisting of an input object
         (typically a vector) and a desired output value (also called the supervisory signal).
         A supervised learning algorithm analyzes the training data and produces an inferred function,
         which can be used for mapping new examples. An optimal scenario will allow for the
         algorithm to correctly determine the class labels for unseen instances. This requires
         the learning algorithm to generalize from the training data to unseen situations in a
         'reasonable' way (see inductive bias).
''', keyphrase_ngram_range=(1, 3))

CPU times: total: 2.5 s
Wall time: 323 ms


In [20]:
complaints = pd.read_csv("./complaints/complaints.csv").dropna().head(3000)

C:\Users\USER_ELISEY\AppData\Local\Temp\ipykernel_18704\3186319543.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints = pd.read_csv("./complaints/complaints.csv").dropna().head(3000)


In [24]:
complaints = complaints.rename(columns={"Consumer complaint narrative": "narrative"})

In [26]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [30]:
def f(text):
    from keybert import KeyBERT
    return [i[0] for i in KeyBERT().extract_keywords(text, keyphrase_ngram_range=(1, 3))]

In [31]:
f(complaints.iloc[0].narrative)

['termination fees rent',
 'given notice vacate',
 'early termination fees',
 'termination fees',
 'notice vacate']

In [34]:
complaints["narrative_keyngrams"] = complaints["narrative"].parallel_apply(f)

In [35]:
complaints.head(5)

,Date received,Product,Sub-product,Issue,Sub-issue,narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,narrative_keyngrams
6880,2016-02-16,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,I was given a notice to vacate XX/XX/2015. I p...,Company believes the complaint is the result o...,"HW Holding, Inc",FL,34711.0,Servicemember,Consent provided,Web,2016-02-19,Closed with explanation,Yes,No,1788003,"[termination fees rent, given notice vacate, e..."
6931,2015-11-08,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,I called this Revenue Recovery to ask for an i...,Company chooses not to provide a public response,Revenue Recovery Corporation,KY,40272.0,Servicemember,Consent provided,Web,2015-11-08,Closed with explanation,Yes,No,1645453,"[collections account credit, credit report col..."
7086,2016-10-26,Debt collection,Payday loan,Improper contact or sharing of info,Contacted me after I asked not to,Company called Premier Portfolio Group continu...,Company believes it acted appropriately as aut...,Premier Portfolio Group,SD,57702.0,Servicemember,Consent provided,Web,2016-10-28,Closed with explanation,Yes,No,2180605,"[phone numbers company, numbers company calls,..."
7107,2016-10-28,Debt collection,I do not know,Taking/threatening an illegal action,Sued where didn't live/sign for debt,The firm of Pressler & Pressler has filed a le...,Company believes it acted appropriately as aut...,"Pressler & Pressler, LLP",NJ,7726.0,Servicemember,Consent provided,Web,2016-10-28,Closed with explanation,Yes,No,2183670,"[bank contacting pressler, contacted claimed d..."
7125,2016-02-04,Debt collection,Credit card,Taking/threatening an illegal action,Attempted to/Collected exempt funds,I became behind on my Military Star Card when ...,Company believes it acted appropriately as aut...,Army and Air Force Exchange Service,VA,22554.0,Servicemember,Consent provided,Web,2016-02-04,Closed with explanation,Yes,Yes,1774097,"[told filing hardship, filing hardship, bills ..."


In [44]:
all_ngrams = []
complaints["narrative_keyngrams"].apply(lambda x: all_ngrams.extend(x))
#set(all_ngrams)

6880       None
6931       None
7086       None
7107       None
7125       None
           ... 
3249401    None
3249707    None
3250032    None
3250183    None
3250394    None
Name: narrative_keyngrams, Length: 3000, dtype: object

In [ ]:
sns.heatmap(complaints, x="")

In [48]:
import plotly.express as px

In [47]:
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [61]:
dtm = tfidf.fit_transform(complaints['narrative_keyngrams'].apply(lambda x: " ".join(x)))

In [67]:
dtm

<3000x1542 sparse matrix of type '<class 'numpy.float64'>'
	with 22285 stored elements in Compressed Sparse Row format>

In [72]:
bow = CountVectorizer()
bow_vectors = bow.fit_transform(complaints['narrative_keyngrams'].apply(lambda x: " ".join(x)))

In [71]:
bow_vectors

<3000x3047 sparse matrix of type '<class 'numpy.int64'>'
	with 23790 stored elements in Compressed Sparse Row format>